In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
import os
import datetime
from statsmodels.stats.weightstats import ttest_ind
import scipy
from geopy.distance import geodesic

In [2]:
%load_ext google.cloud.bigquery

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="sunny-footing-321404-513454d4be38.json"

# Difference in mean duration of rides between male and female riders: SF

In [4]:
client = bigquery.Client()

query = """
    SELECT member_gender, duration_sec FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
    WHERE (member_gender = 'Female' OR member_gender = 'Male') AND start_date >= '2017-01-01 00:00:00'
"""

query_job = client.query(query)

In [5]:
sf_times = query_job.to_dataframe()
sf_times['duration_minutes'] = sf_times['duration_sec'].apply(lambda x: x / 60)
sf_times

,member_gender,duration_sec,duration_minutes
0,Male,2646,44.100000
1,Male,626,10.433333
2,Male,297,4.950000
3,Male,127,2.116667
4,Male,2179,36.316667
...,...,...,...
850968,Female,637,10.616667
850969,Female,1697,28.283333
850970,Female,442,7.366667
850971,Female,2396,39.933333


In [6]:
sf_female_durations = sf_times[sf_times['member_gender'] == 'Female']['duration_minutes']
sf_male_durations = sf_times[sf_times['member_gender'] == 'Male']['duration_minutes']

diff_of_means = np.mean(sf_female_durations) - np.mean(sf_male_durations)
s_e = np.sqrt((np.std(sf_female_durations)**2/len(sf_female_durations)) + (np.std(sf_male_durations)**2/len(sf_male_durations)))
t_star = scipy.stats.t.ppf(q=1-.05/2,df=min(len(sf_female_durations), len(sf_male_durations)))
CI = (diff_of_means - t_star*s_e, diff_of_means + t_star*s_e)
CI

(3.3237005861850406, 3.762211531391973)

The results are significant. There is enough evidence to say that the average duration of female riders in san franscisco is more than the average duration of male riders in san franscisco.

# Difference in mean duration of rides between male and female riders: NY

In [7]:
client = bigquery.Client()

query = """
    SELECT gender, tripduration FROM `bigquery-public-data.new_york_citibike.citibike_trips`
    WHERE (gender = 'male' OR gender = 'female') AND starttime >= '2017-01-01 00:00:00'
"""

query_job = client.query(query) 

In [8]:
ny_times = query_job.to_dataframe()
ny_times['duration_minutes'] = ny_times['tripduration'].apply(lambda x: x / 60)
ny_times

,gender,tripduration,duration_minutes
0,male,607,10.116667
1,male,920,15.333333
2,male,676,11.266667
3,male,185,3.083333
4,male,383,6.383333
...,...,...,...
17760096,female,1367,22.783333
17760097,female,361,6.016667
17760098,female,1350,22.500000
17760099,female,245,4.083333


In [9]:
ny_female_durations = ny_times[ny_times['gender'] == 'female']['duration_minutes']
ny_male_durations = ny_times[ny_times['gender'] == 'male']['duration_minutes']

diff_of_means = np.mean(ny_female_durations) - np.mean(ny_male_durations)
s_e = np.sqrt((np.std(ny_female_durations)**2/len(ny_female_durations)) + (np.std(ny_male_durations)**2/len(ny_male_durations)))
t_star = scipy.stats.t.ppf(q=1-.05/2,df=min(len(ny_female_durations), len(ny_male_durations)))
CI = (diff_of_means - t_star*s_e, diff_of_means + t_star*s_e)
CI

(2.3418808592876825, 3.0669868558315176)

The results are significant. There is enough evidence to say that the average duration of female riders in NYC is more than the average duration of male riders in NYC.

# Difference in mean duration of rides of female riders between SF and NY

In [10]:
diff_of_means = np.mean(sf_female_durations) - np.mean(ny_female_durations)
s_e = np.sqrt((np.std(sf_female_durations)**2/len(sf_female_durations)) + (np.std(ny_female_durations)**2/len(ny_female_durations)))
t_star = scipy.stats.t.ppf(q=1-.05/2,df=min(len(sf_female_durations), len(ny_female_durations)))
CI = (diff_of_means - t_star*s_e, diff_of_means + t_star*s_e)
CI

(-1.5882716235986893, -0.8191422721111192)

The results are significant. There is enough evidence to say that the average duration of female riders in SF is less than the average duration of female riders in NYC.

# Difference in mean duration of rides of male riders between SF and NY

In [11]:
diff_of_means = np.mean(sf_male_durations) - np.mean(ny_male_durations)
s_e = np.sqrt((np.std(sf_male_durations)**2/len(sf_male_durations)) + (np.std(ny_male_durations)**2/len(ny_male_durations)))
t_star = scipy.stats.t.ppf(q=1-.05/2,df=min(len(sf_male_durations), len(ny_male_durations)))
CI = (diff_of_means - t_star*s_e, diff_of_means + t_star*s_e)
CI

(-2.220074330925362, -1.8643839672422597)

The results are significant. There is enough evidence to say that the average duration of male riders in SF is less than the average duration of male riders in NYC.

# Difference in mean distance of rides between male and female riders: SF

Note that this is finding shortest distance between two points, we do not know the actual distance traveled by the bike.

In [12]:
client = bigquery.Client()

query = """
    SELECT member_gender, start_station_latitude, start_station_longitude, end_station_latitude, end_station_longitude
    FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
    WHERE (member_gender = 'Female' OR member_gender = 'Male') AND start_date >= '2017-01-01 00:00:00'
"""

query_job = client.query(query)

In [13]:
sf_distances = query_job.to_dataframe()
distances = []
for start_lat, start_long, end_lat, end_long in zip(sf_distances['start_station_latitude'], 
                                                    sf_distances['start_station_longitude'],
                                                    sf_distances['end_station_latitude'],
                                                    sf_distances['end_station_longitude']):
    start = (start_lat, start_long)
    end = (end_lat, end_long)
    distances.append(geodesic(start, end).mi)
sf_distances['distances'] = distances
sf_distances

,member_gender,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,distances
0,Male,37.769305,-122.426826,37.783988,-122.412408,1.283864
1,Male,37.868813,-122.258764,37.872355,-122.266447,0.485934
2,Female,37.763847,-122.413004,37.759200,-122.409851,0.363995
3,Male,37.761420,-122.426435,37.761420,-122.426435,0.000000
4,Male,37.839649,-122.271756,37.805318,-122.294837,2.683352
...,...,...,...,...,...,...
850968,Male,37.771058,-122.402717,37.786375,-122.404904,1.063157
850969,Male,37.771058,-122.402717,37.786375,-122.404904,1.063157
850970,Male,37.771058,-122.402717,37.786375,-122.404904,1.063157
850971,Male,37.771058,-122.402717,37.786375,-122.404904,1.063157


In [14]:
sf_female_distances = sf_distances[sf_distances['member_gender'] == 'Female']['distances']
sf_male_distances = sf_distances[sf_distances['member_gender'] == 'Male']['distances']

diff_of_means = np.mean(sf_female_distances) - np.mean(sf_male_distances)
s_e = np.sqrt((np.std(sf_female_distances)**2/len(sf_female_distances)) + (np.std(sf_male_distances)**2/len(sf_male_distances)))
t_star = scipy.stats.t.ppf(q=1-.05/2,df=min(len(sf_female_distances), len(sf_male_distances)))
CI = (diff_of_means - t_star*s_e, diff_of_means + t_star*s_e)
CI

(0.05625778200163658, 0.06263193845890089)

The results are significant. There is enough evidence to say that the average distance of female riders in SF is more than the average distance of male riders in SF.

# Difference in mean distance of rides between male and female riders: NY

In [15]:
client = bigquery.Client()

query = """
    SELECT gender, start_station_latitude, start_station_longitude, end_station_latitude, end_station_longitude
    FROM `bigquery-public-data.new_york_citibike.citibike_trips`
    WHERE (gender = 'male' OR gender = 'female') AND starttime >= '2017-01-01 00:00:00'
"""

query_job = client.query(query) 

In [16]:
ny_distances = query_job.to_dataframe()
distances = []
for start_lat, start_long, end_lat, end_long in zip(ny_distances['start_station_latitude'], 
                                                    ny_distances['start_station_longitude'],
                                                    ny_distances['end_station_latitude'],
                                                    ny_distances['end_station_longitude']):
    start = (start_lat, start_long)
    end = (end_lat, end_long)
    distances.append(geodesic(start, end).mi)
ny_distances['distances'] = distances
ny_distances

,gender,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,distances
0,male,40.756405,-73.990026,40.766697,-73.990617,0.710801
1,male,40.756405,-73.990026,40.760301,-73.998842,0.535002
2,male,40.756405,-73.990026,40.737815,-73.999947,1.384404
3,male,40.756405,-73.990026,40.718710,-74.009001,2.785231
4,male,40.756405,-73.990026,40.771522,-73.990541,1.043433
...,...,...,...,...,...,...
17760096,female,40.742754,-74.007474,40.746920,-74.004519,0.326606
17760097,female,40.790179,-73.972889,40.765909,-73.976342,1.684474
17760098,female,40.732618,-73.991580,40.730473,-73.986724,0.294773
17760099,female,40.735367,-73.987974,40.720828,-73.977932,1.133274


In [17]:
ny_female_distances = ny_distances[ny_distances['gender'] == 'female']['distances']
ny_male_distances = ny_distances[ny_distances['gender'] == 'male']['distances']

diff_of_means = np.mean(ny_female_distances) - np.mean(ny_male_distances)
s_e = np.sqrt((np.std(ny_female_distances)**2/len(ny_female_distances)) + (np.std(ny_male_distances)**2/len(ny_male_distances)))
t_star = scipy.stats.t.ppf(q=1-.05/2,df=min(len(ny_female_distances), len(ny_male_distances)))
CI = (diff_of_means - t_star*s_e, diff_of_means + t_star*s_e)
CI

(0.043243550931630396, 0.06014290012231178)

The results are significant. There is enough evidence to say that the average distance of female riders in NY is more than the average distance of male riders in NY.

# Difference in mean distance of rides of female riders between SF and NY

In [18]:
diff_of_means = np.mean(sf_female_distances) - np.mean(ny_female_distances)
s_e = np.sqrt((np.std(sf_female_distances)**2/len(sf_female_distances)) + (np.std(ny_female_distances)**2/len(ny_female_distances)))
t_star = scipy.stats.t.ppf(q=1-.05/2,df=min(len(sf_female_distances), len(ny_female_distances)))
CI = (diff_of_means - t_star*s_e, diff_of_means + t_star*s_e)
CI

(-0.14142866704960513, -0.12616126672928693)

The results are significant. There is enough evidence to say that the average distance of female riders in SF is less than the average distance of female riders in NY.

# Difference in mean distance of rides of male riders between SF and NY

In [19]:
diff_of_means = np.mean(sf_male_distances) - np.mean(ny_male_distances)
s_e = np.sqrt((np.std(sf_male_distances)**2/len(sf_male_distances)) + (np.std(ny_male_distances)**2/len(ny_male_distances)))
t_star = scipy.stats.t.ppf(q=1-.05/2,df=min(len(sf_male_distances), len(ny_male_distances)))
CI = (diff_of_means - t_star*s_e, diff_of_means + t_star*s_e)
CI

(-0.14637171537576057, -0.1367214878097268)

The results are significant. There is enough evidence to say that the average distance of male riders in SF is less than the average distance of male riders in NY.

# Difference in proportion of female riders between cities

In [20]:
client = bigquery.Client()

query = """
    SELECT member_gender FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
    WHERE (member_gender = 'Female' OR member_gender = 'Male') AND start_date >= '2017-01-01 00:00:00'
"""

query_job = client.query(query) 

In [21]:
sf_genders = query_job.to_dataframe()
sf_genders.value_counts()

member_gender
Male             651771
Female           199202
dtype: int64

In [22]:
client = bigquery.Client()

query = """
    SELECT gender FROM `bigquery-public-data.new_york_citibike.citibike_trips`
    WHERE (gender = 'male' OR gender = 'female') AND starttime >= '2017-01-01 00:00:00'
"""

query_job = client.query(query) 

In [23]:
ny_genders = query_job.to_dataframe()
ny_genders.value_counts()

gender
male      13262473
female     4497628
dtype: int64

In [24]:
data = pd.DataFrame(columns = ['SF', 'NY'])
data['SF'] = list(sf_genders.value_counts())
data['NY'] = list(ny_genders.value_counts())
data.index = ['male', 'female']
data

,SF,NY
male,651771,13262473
female,199202,4497628


In [25]:
import numpy as np
# population proportion differences (proportion of females in SF and NY)
# just realized should be filtered by 2017 and after
p_female_SF = data.loc['female', 'SF'] / data['SF'].sum()
p_female_NY = data.loc['female', 'NY'] / data['NY'].sum()
n_SF = data['SF'].sum()
n_NY = data['NY'].sum()

p_hat = p_female_SF - p_female_NY
s_e = np.sqrt((p_female_SF * (1 - p_female_SF) / n_SF) + (p_female_NY * (1 - p_female_NY) / n_NY))
CI = (p_hat - 1.96*s_e, p_hat + 1.96*s_e)

In [26]:
CI

(-0.020078157118831246, -0.01823393622132637)

The results are significant. There is enough evidence to say that the proportion of female riders in san franscisco is less than the proportion of female riders in New York.

# Difference in proportion of generations between cities

In [27]:
# map year to generation
def map_year_to_generation(year):
    if 1928 <= year <= 1945:
        generation = 'Silent'
    elif 1946 <= year <= 1964:
        generation = 'Baby Boomers'
    elif 1965 <= year <= 1980:
        generation = 'Generation X'
    elif 1981 <= year <= 1996:
        generation = 'Millennials'
    elif 1997 <= year:
        generation = 'Generation Z'
    else:
        generation = 'Before Silent'
    return generation

In [28]:
client = bigquery.Client()

query = """
    SELECT member_birth_year FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
    WHERE start_date >= '2017-01-01 00:00:00'
"""

query_job = client.query(query) 

In [29]:
sf_generations = query_job.to_dataframe()
sf_generations['generation'] = sf_generations['member_birth_year'].apply(lambda x: map_year_to_generation(x))
sf_generations

,member_birth_year,generation
0,1954.0,Baby Boomers
1,1954.0,Baby Boomers
2,1954.0,Baby Boomers
3,1952.0,Baby Boomers
4,1948.0,Baby Boomers
...,...,...
963766,1997.0,Generation Z
963767,1997.0,Generation Z
963768,1997.0,Generation Z
963769,1997.0,Generation Z


In [30]:
sf_generations['generation'].value_counts().sort_index()

Baby Boomers      68635
Before Silent    101906
Generation X     259685
Generation Z      12827
Millennials      518330
Silent             2388
Name: generation, dtype: int64

In [31]:
client = bigquery.Client()

query = """
    SELECT birth_year FROM `bigquery-public-data.new_york_citibike.citibike_trips`
    WHERE starttime >= '2017-01-01 00:00:00'
"""

query_job = client.query(query) 

In [32]:
ny_generations = query_job.to_dataframe()
ny_generations['generation'] = ny_generations['birth_year'].apply(lambda x: map_year_to_generation(x))
ny_generations

,birth_year,generation
0,1995.0,Millennials
1,1995.0,Millennials
2,1995.0,Millennials
3,1995.0,Millennials
4,1995.0,Millennials
...,...,...
19789697,1969.0,Generation X
19789698,1969.0,Generation X
19789699,1969.0,Generation X
19789700,1969.0,Generation X


In [33]:
ny_generations['generation'].value_counts().sort_index()

Baby Boomers     2423551
Before Silent    1531801
Generation X     6106357
Generation Z      309690
Millennials      9350028
Silent             68275
Name: generation, dtype: int64

In [34]:
# get rid of birth years before the silent generation to get rid of unrealistic data
data = pd.DataFrame(columns = ['SF', 'NY'])
data['SF'] = list(sf_generations['generation'].value_counts().sort_index())
data['NY'] = list(ny_generations['generation'].value_counts().sort_index())
data.index = ['Baby Boomers', 'Before Silent', 'Generation X', 'Generation Z', 'Millennials', 'Silent']
data = data.drop(index = ['Before Silent'])
data

,SF,NY
Baby Boomers,68635,2423551
Generation X,259685,6106357
Generation Z,12827,309690
Millennials,518330,9350028
Silent,2388,68275


In [35]:
data_proportions = data
data_proportions['SF'] = data_proportions['SF']/data_proportions['SF'].sum()
data_proportions['NY'] = data_proportions['NY']/data_proportions['NY'].sum()
data_proportions

,SF,NY
Baby Boomers,0.079635,0.132740
Generation X,0.301306,0.334450
Generation Z,0.014883,0.016962
Millennials,0.601405,0.512109
Silent,0.002771,0.003739


# Boomers

In [36]:
p_boomers_SF = data.loc['Baby Boomers', 'SF'] / data['SF'].sum()
p_boomers_NY = data.loc['Baby Boomers', 'NY'] / data['NY'].sum()
n_SF = data['SF'].sum()
n_NY = data['NY'].sum()

p_hat = p_boomers_SF - p_boomers_NY
s_e = np.sqrt((p_boomers_SF * (1 - p_boomers_SF) / n_SF) + (p_boomers_NY * (1 - p_boomers_NY) / n_NY))
CI = (p_hat - 1.96*s_e, p_hat + 1.96*s_e)

In [37]:
CI

(-0.9038745030213239, 0.7976656909000309)

The results are significant. There is enough evidence to say that the proportion of baby boomer riders in san franscisco is less than the proportion of baby boomer riders in New York.

# Gen X

In [38]:
p_X_SF = data.loc['Generation X', 'SF'] / data['SF'].sum()
p_X_NY = data.loc['Generation X', 'NY'] / data['NY'].sum()
n_SF = data['SF'].sum()
n_NY = data['NY'].sum()

p_hat = p_X_SF - p_X_NY
s_e = np.sqrt((p_X_SF * (1 - p_X_SF) / n_SF) + (p_X_NY * (1 - p_X_NY) / n_NY))
CI = (p_hat - 1.96*s_e, p_hat + 1.96*s_e)

In [39]:
CI

(-1.323047433622754, 1.2567589913305182)

The results are significant. There is enough evidence to say that the proportion of generation x riders in san franscisco is less than the proportion of generation x riders in New York.

# Generation Z

In [40]:
p_Z_SF = data.loc['Generation Z', 'SF'] / data['SF'].sum()
p_Z_NY = data.loc['Generation Z', 'NY'] / data['NY'].sum()
n_SF = data['SF'].sum()
n_NY = data['NY'].sum()

p_hat = p_Z_SF - p_Z_NY
s_e = np.sqrt((p_Z_SF * (1 - p_Z_SF) / n_SF) + (p_Z_NY * (1 - p_Z_NY) / n_NY))
CI = (p_hat - 1.96*s_e, p_hat + 1.96*s_e)

In [41]:
CI

(-0.34903570147060814, 0.34487744063804704)

The results are significant. There is enough evidence to say that the proportion of generation z riders in san franscisco is less than the proportion of generation z riders in New York.

# Millennials

In [42]:
p_millennial_SF = data.loc['Millennials', 'SF'] / data['SF'].sum()
p_millennial_NY = data.loc['Millennials', 'NY'] / data['NY'].sum()
n_SF = data['SF'].sum()
n_NY = data['NY'].sum()

p_hat = p_millennial_SF - p_millennial_NY
s_e = np.sqrt((p_millennial_SF * (1 - p_millennial_SF) / n_SF) + (p_millennial_NY * (1 - p_millennial_NY) / n_NY))
CI = (p_hat - 1.96*s_e, p_hat + 1.96*s_e)

In [43]:
CI

(-1.2821019139578087, 1.4606949119978714)

The results are significant. There is enough evidence to say that the proportion of millennial riders in san franscisco is more than the proportion of millennial riders in New York.

# Silent

In [44]:
p_silent_SF = data.loc['Silent', 'SF'] / data['SF'].sum()
p_silent_NY = data.loc['Silent', 'NY'] / data['NY'].sum()
n_SF = data['SF'].sum()
n_NY = data['NY'].sum()

p_hat = p_silent_SF - p_silent_NY
s_e = np.sqrt((p_silent_SF * (1 - p_silent_SF) / n_SF) + (p_silent_NY * (1 - p_silent_NY) / n_NY))
CI = (p_hat - 1.96*s_e, p_hat + 1.96*s_e)

In [45]:
CI

(-0.1588497749958343, 0.1569122922018617)

The results are significant. There is enough evidence to say that the proportion of silent riders in san franscisco is less than the proportion of silent riders in New York.